# Notes, 9/6/18

## Mac homebrew

### Build from source

* This can be used to make sure your libraries have been compiled using the same compilers that you are going to use for JEDI

* Need to do this on the mac since the default compilers are clang
    * These have access to gcc, but it's an ancient version (4.x)
    
* Add the --build-from-source option to the brew install command
    * --HEAD will check out a version from a repository
    * you can define "head" in the brew formula which gives you the ability to define a particular branch or tag to use

### Edit a formula

* Formulas are ruby scripts
    * To set an environment variable in a formula
        * ENV\['FC'\] = '/usr/local/bin/gfortran-7'

* brew edit <formula\>


* After finishing you can check your formula by "auditing"
    * brew audit <formula\>
    * This checks for acceptability to merge your formula, or edits to and existing formula into the brew repository

### Create your own formula

* Need source code
    * Brew wants source code so that it can be demonstrated that you can build on your platform

* Two ways to provide source code:
    * Tar file
        * Give brew path to a tarball containing the source code and build configuration
        * Can use autoconf, cmake, etc.
        * Give brew the sha256 sum for the tar file
            * shasum -a256 <tar_file\>
    * Repository
        * Give brew url for cloning
        * Can specify particular branches or tags
        * Give brew the sha1 value for the commit you are checking out

* Create a new formula
    * brew edit <formula\>
    
### Environment for homebrew

* export HOMEBREW_CC="gcc-7"
* export HOMEBREW_CXX="g++-7"

* brew --env
* brew config

### Building boost

* brew install --verbose --build-from-source --c++11 boost
   * says it is ignoring c++11, but it uses c++1 anyway
   * c++11 is necessary to get boost name mangling to sync up with JEDI name mangling

### Formulas for open-mpi, hdf5, netcdf

* brew edit open-mpi, hdf5, netcdf
    * Change depends_on "gcc" to "gcc@7"
    * For Fortran compiling, add: ENV\['FC'\] = '/usr/local/bin/gfortran:q'
    
* For open-mpi only
    * brew edit open-mpi
        * add to the head section
            * :using =\> :git,
            * :branch =\> "v3.2.1",
            * :revision =\> "<full sha value from git log\>"
        * ENV\['CC'\] = '/usr/local/bin/gcc-7'
        * ENV\['CXX'\] = '/usr/local/bin/g++-7'

* For netcdf only
    * do a brew edit to see the url for downloading netcdf-fortran source tarball
    * do a wget to download this tarball, unpack it, and edit the top level CMakeLists.txt
        * get rid of the macro-backtrace options to gfortran (these are for clang, not gcc)
    * repack the tar ball, and store in a safe place
    * run shasum -a256 on the tarball
    * brew edit netcdf
        * in resource "fortran" do
            * comment out the specs and replace with url and sha256 values in accordance with the new tarball
                * use the file://<full path\> form of the url

* brew install --verbose --build-from-source open-mpi, hdf5, netcdf
   * Add the --c++11 option for netcdf
   * Don't use the --HEAD option for open-mpi
        
### Formulas for ecbuild, eckit, fckit

* Need to create new formulas for these

* brew create ecbuild, eckit, fckit

* For ecbuild:
~~~~~~~~
class Ecbuild < Formula
  desc "CMake macros for building JEDI"
  homepage "https://github.com/ECMWF/"
  url "https://github.com/ECMWF/ecbuild.git",
    :using => :git,
    :tag => "2.9.0",
    :revision => "10d46b3d22df192405aa4da148e5c80bd7a814e0"

  depends_on "cmake" => :build

  def install
    mkdir "build" do
      system "cmake", "..", *std_cmake_args
      system "make", "install"
    end
  end

  test do
  end
end
~~~~~~~~

* For eckit:
~~~~~~~~
class Eckit < Formula
  desc "C++ utilities for JEDI"
  homepage "https://github.com/ECMWF"
  url "https://github.com/ECMWF/eckit.git",
    :using => :git,
    :tag => "0.22.0",
    :revision => "a1d8af4a48cdfc9088761a2ddf69eceb69e4ba13"

  depends_on "cmake" => :build
  depends_on "ecbuild"
  depends_on "eigen"
  depends_on "open-mpi"

  def install
    # Allow open-mpi to oversubscribe processes since
    # only have 2 cores on this system.
    ENV['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
    mkdir "build" do
      system "ecbuild", "..", *std_cmake_args
      system "make"
      system "ctest"
      system "make", "install"
    end
  end

  test do
  end
end
~~~~~~~~

* For fckit:
~~~~~~~~
class Fckit < Formula
  desc "Fortran utilities for JEDI"
  homepage "https://github.com/ECMWF/"
  url "https://github.com/ECMWF/fckit.git",
    :using => :git,
    :tag => "0.5.2",
    :revision => "dad53e01cb6353476ca9c890071713dca353da10"

  depends_on "cmake" => :build
  depends_on "ecbuild"
  depends_on "eckit"

  def install
    # Allow open-mpi to oversubscribe processes since
    # only have 2 cores on this system.
    ENV['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
    mkdir "build" do
      system "ecbuild", "..", *std_cmake_args
      system "make"
      system "ctest"
      system "make", "install"
    end
  end

  test do
  end
end
~~~~~~~~

* To install: brew install --verbose ecbuild, eckit, fckit

